In [61]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data.dataset import Subset
from torchvision import datasets
from torchvision.transforms import ToTensor
import sys
import os
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import KFold
import math
import copy
from functools import partial
import tempfile
import numpy as np
import time

In [13]:
os.chdir("C:\\Users\\tjjtj\\CG4002\\glove_data")
print(os.getcwd())

C:\Users\tjjtj\CG4002\glove_data


In [15]:
def add_sum_features(input):
    temp = input
    for i in range(6):
        curr_col = i * 4
        mean = torch.mean(temp[:, curr_col]).item()
        min = torch.min(temp[:, curr_col]).item()
        max = torch.max(temp[:, curr_col]).item()
        mean_col = torch.full((50, 1), mean)
        min_col = torch.full((50, 1), min)
        max_col = torch.full((50, 1), max)

        left = temp[:, :curr_col+1]
        right = temp[:, curr_col+1:]
        temp = torch.cat((left, mean_col, min_col, max_col, right), dim=1)
    return temp

In [19]:
class CustomDataset(Dataset):
    def __init__(self, labels_file, data_dir, means=None, std_devs=None):
        if (data_dir == ""):
            path = "C:\\Users\\tjjtj\\CG4002\\glove_data"
        else:
            path = "C:\\Users\\tjjtj\\CG4002\\glove_data\\" + data_dir
        path_to_labels_file = path + "\\" + labels_file
        self.data_labels = pd.read_csv(path_to_labels_file, header=None)

        datas = []
        self.datapoints = []
        for i in range(len(self.data_labels)):
            file_path = os.path.join(path, self.data_labels.iloc[i, 0])
            
            with open(file_path, encoding="utf-8") as f:
                file_content = f.read()
            line_array = []
            lines = file_content.strip().split("\n")
            for line in lines:
                num_array = []
                nums = line.strip().split(",")
                for num in nums:
                    num_array.append(float(num))
                line_array.append(num_array)
            temp = torch.FloatTensor(line_array)
            data = add_sum_features(temp)

            label = self.data_labels.iloc[i, 1]

            datapoint = [data, label]
            datas.append(data)
            self.datapoints.append(datapoint)

        datas = torch.stack(datas)
        
        if (means is None):
            means = []
            for i in range(24):
                means.append(torch.mean(datas[:, :, i]).item())
        if (std_devs is None):
            std_devs = []
            for i in range(24):
                std_devs.append(torch.std(datas[:, :, i]).item())
        self.means = means
        self.std_devs = std_devs            
        
        for datapoint in self.datapoints:
            data = datapoint[0]
            for i in range(24):
                curr_data = data[:, i]
                curr_data = torch.sub(curr_data, means[i])
                curr_data = torch.div(curr_data, std_devs[i])
                datapoint[0][:, i] = curr_data
        
        self.path = path

    def __len__(self):
        return len(self.data_labels)

    def __getitem__(self, idx):
        data = self.datapoints[idx][0]
        label = self.datapoints[idx][1]
        return data, label

In [21]:
def create_dataloaders(dataset, batch_size):
    train_size = int(len(dataset) * 0.8)
    train_set, valid_set = random_split(dataset, [train_size, len(dataset) - train_size])
    trainloader = DataLoader(train_set, batch_size, shuffle=True)
    validloader = DataLoader(valid_set, batch_size, shuffle=True)
    return trainloader, validloader

In [23]:
class Net(nn.Module):

    def __init__(self, hidden_size=14):
        super(Net, self).__init__()
        self.rnn = nn.RNN(25, hidden_size, batch_first = True)
        self.classifier = nn.Linear(hidden_size, 8)

    def forward(self, x):
        _, x = self.rnn(x)
        x = self.classifier(x)
        output = x
        return output

In [25]:
# load saved model
net = Net(100)
net.load_state_dict(torch.load("95%_train_acc_log_12_model_1_100_hidden.pt", weights_only=True))
net.eval()

Net(
  (rnn): RNN(25, 100, batch_first=True)
  (classifier): Linear(in_features=100, out_features=8, bias=True)
)

In [27]:
params = list(net.named_parameters())
with open("model_description.txt", "w") as f:
    for param in params:
        param_name = param[0]
        f.write(param_name)
        f.write(" ")
        f.write(str(param[1].size()))
        f.write("\n")
        f.write(str(param[1].tolist()))
        f.write("\n")
        if ("weight" in param_name):
            f.write(param_name + "_transposed")
            f.write(" ")
            f.write(str(param[1].T.size()))
            f.write("\n")
            f.write(str(param[1].T.tolist()))
            f.write("\n")

In [29]:
with open("model_description.txt", "r") as f:
    contents = f.read()

contents = contents.replace("[", "{")
contents = contents.replace("]", "}")

with open("model_description_cpp.txt", "w") as f:
    f.write(contents)

In [63]:
with open("model_description_cpp.txt", "r") as f:
    contents = f.read()
    contents_list = contents.strip().split("\n")
    RNN_INPUT_WEIGHTS_TRANSPOSED = contents_list[3]
    RNN_HIDDEN_WEIGHTS_TRANSPOSED = contents_list[7]
    RNN_INPUT_BIASES = contents_list[9]
    RNN_HIDDEN_BIASES = contents_list[11]
    CLASSIFIER_WEIGHTS_TRANSPOSED = contents_list[15]
    CLASSIFIER_BIASES = contents_list[17]

with open("log_12.txt", "r") as f:
    contents = f.read()
    contents_list = contents.strip().split("\n")
    for row in contents_list:
        row_list = row.split()
        if row_list[0] == "Means":
            MEANS = row[row.find("["):]
            MEANS = MEANS.replace("[", "{")
            MEANS = MEANS.replace("]", "}")
        elif row_list[0] == "Std" and row_list[1] == "devs":
            STD_DEVS = row[row.find("["):]
            STD_DEVS = STD_DEVS.replace("[", "{")
            STD_DEVS = STD_DEVS.replace("]", "}")
            break

SEQ_LEN = 50
NUM_INPUT_FEATURES = 7
NUM_FEATURES = 25
HIDDEN_SIZE = 100
NUM_CLASSES = 8

with open("C:\\Users\\tjjtj\\Desktop\\NUS\\Y4S1\\CG4002\\Project\\NN_Vitis\\hls_component\\consts.h", "w") as f:
    f.write(f"const int SEQ_LEN = {SEQ_LEN};\n")
    f.write(f"const int NUM_INPUT_FEATURES = {NUM_INPUT_FEATURES};\n")
    f.write(f"const int NUM_FEATURES = {NUM_FEATURES};\n")
    f.write(f"const int HIDDEN_SIZE = {HIDDEN_SIZE};\n")
    f.write(f"const int NUM_CLASSES = {NUM_CLASSES};\n")
    f.write("\n")
    f.write(f"const float RNN_INPUT_WEIGHTS_TRANSPOSED[NUM_FEATURES][HIDDEN_SIZE] = {RNN_INPUT_WEIGHTS_TRANSPOSED};\n")
    f.write(f"float RNN_INPUT_BIASES[HIDDEN_SIZE] = {RNN_INPUT_BIASES};\n")
    f.write(f"const float RNN_HIDDEN_WEIGHTS_TRANSPOSED[HIDDEN_SIZE][HIDDEN_SIZE] = {RNN_HIDDEN_WEIGHTS_TRANSPOSED};\n")
    f.write(f"float RNN_HIDDEN_BIASES[HIDDEN_SIZE] = {RNN_HIDDEN_BIASES};\n")
    f.write(f"const float CLASSIFIER_WEIGHTS_TRANSPOSED[HIDDEN_SIZE][NUM_CLASSES] = {CLASSIFIER_WEIGHTS_TRANSPOSED};\n")
    f.write(f"float CLASSIFIER_BIASES[NUM_CLASSES] = {CLASSIFIER_BIASES};\n")
    f.write(f"float MEANS[NUM_FEATURES - 1] = {MEANS};\n")
    f.write(f"float STD_DEVS[NUM_FEATURES - 1] = {STD_DEVS};\n")

In [33]:
os.chdir("combined_data")

In [35]:
def write_test_input_file(gesture):
    labels_file = gesture + "_labels.csv"
    with open(labels_file, "r") as f:
        content = f.read()
        first_path = content.strip().split()[0][:-1]
        label = content.strip().split()[1]

    with open(first_path, "r") as f:
        contents = f.read()
        contents_list = contents.split()
        contents_array = np.empty((50, 7))
        index = 0 
        for line in contents_list:
            line_list = line.split(",")
            contents_array[index] =  np.array(line_list)
            index += 1    
        shape = str(contents_array.shape)
        
        contents_array = np.array2string(contents_array, separator=", ")
        contents_array = contents_array.replace("\n", "")

    filename = "test_" + gesture + "_input"
    filename_with_extension = filename + ".txt"
    with open(filename_with_extension, "w") as f:
        f.write(shape)
        f.write("\n")
        f.write(label)
        f.write("\n")
        f.write(contents_array)
    
    contents_array = contents_array.replace("[", "{")
    contents_array = contents_array.replace("]", "}")

    cpp_filename_with_extension = filename + "_cpp" + ".txt"
    with open(cpp_filename_with_extension, "w") as f:
        f.write(shape)
        f.write("\n")
        f.write(label)
        f.write("\n")
        f.write(contents_array)
        
gestures = ["stationary", "random", "wave", "punch", "swipe", "wipe", "shake", "hammer"]
for gesture in gestures:
    write_test_input_file(gesture)

In [37]:
def write_expanded_test_input_file(gesture):
    labels_file = gesture + "_labels.csv"
    with open(labels_file, "r") as f:
        content = f.read()
        first_path = content.strip().split()[0][:-1]
        label = content.strip().split()[1]

    with open(first_path, "r") as f:
        contents = f.read()
        contents_list = contents.split()
        contents_array = np.empty((50, 7))
        index = 0 
        for line in contents_list:
            line_list = line.split(",")
            contents_array[index] =  np.array(line_list)
            index += 1

        in_means = np.mean(contents_array, axis=0)
        in_mins = np.min(contents_array, axis=0)
        in_maxes = np.max(contents_array, axis=0)
        for i in range(6):
            curr_col = i * 4
            mean_column = np.full(50, in_means[i])
            min_column = np.full(50, in_mins[i])
            max_column = np.full(50, in_maxes[i])
            contents_array = np.insert(contents_array, curr_col+1, mean_column, axis=1)
            contents_array = np.insert(contents_array, curr_col+2, min_column, axis=1)
            contents_array = np.insert(contents_array, curr_col+3, max_column, axis=1)

        shape = str(contents_array.shape)
        
        contents_array = np.array2string(contents_array, separator=", ", threshold=10000)
        contents_array = contents_array.replace("\n", "")

    filename = "test_expanded_" + gesture + "_input"
    filename_with_extension = filename + ".txt"
    with open(filename_with_extension, "w") as f:
        f.write(shape)
        f.write("\n")
        f.write(label)
        f.write("\n")
        f.write(contents_array)
    
    contents_array = contents_array.replace("[", "{")
    contents_array = contents_array.replace("]", "}")

    cpp_filename_with_extension = filename + "_cpp" + ".txt"
    with open(cpp_filename_with_extension, "w") as f:
        f.write(shape)
        f.write("\n")
        f.write(label)
        f.write("\n")
        f.write(contents_array)
        
gestures = ["stationary", "random", "wave", "punch", "swipe", "wipe", "shake", "hammer"]
for gesture in gestures:
    write_expanded_test_input_file(gesture)

In [39]:
def transpose_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    transposed_matrix = [[matrix[j][i] for j in range(rows)] for i in range(cols)]
    return transposed_matrix

In [55]:
def multiply(a, b):
    def is_matrix(x):
        return isinstance(x[0], list)

    def to_column_matrix(vec):
        return [[x] for x in vec]

    def to_row_matrix(vec):
        return [vec]

    def dot_product(v1, v2):
        return sum(x * y for x, y in zip(v1, v2))

    def matrix_multiply(A, B):
        rows_A, cols_A = len(A), len(A[0])
        rows_B, cols_B = len(B), len(B[0])
        
        if cols_A != rows_B:
            raise ValueError("Incompatible dimensions for multiplication")

        result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
        for i in range(rows_A):
            for j in range(cols_B):
                for k in range(cols_A):
                    result[i][j] += A[i][k] * B[k][j]
        return result

    a_is_matrix = is_matrix(a)
    b_is_matrix = is_matrix(b)

    if not a_is_matrix and not b_is_matrix:
        if len(a) != len(b):
            raise ValueError("Vectors must be the same length for dot product")
        return dot_product(a, b)

    if not a_is_matrix:
        a = to_row_matrix(a)
    if not b_is_matrix:
        b = to_column_matrix(b)

    result = matrix_multiply(a, b)

    if len(result) == 1 and len(result[0]) == 1:
        return result[0][0]
    elif len(result) == 1:
        return result[0]
    elif len(result[0]) == 1:
        return [row[0] for row in result]
    else:
        return result

In [57]:
def elementwise_tanh(data):
    if isinstance(data[0], list):
        return [[math.tanh(x) for x in row] for row in data]
    else:
        return [math.tanh(x) for x in data]

In [59]:
def add_vectors(vector_A, vector_B):
    result = [0 for _ in range(len(vector_A))]
    for i in range(len(vector_A)):
        result[i]= vector_A[i] + vector_B[i]
    return result

In [47]:
def add_sum_features(input):
    temp = input
    for i in range(6):
        curr_col = i * 4
        mean = torch.mean(temp[:, curr_col]).item()
        min = torch.min(temp[:, curr_col]).item()
        max = torch.max(temp[:, curr_col]).item()
        mean_col = torch.full((50, 1), mean)
        min_col = torch.full((50, 1), min)
        max_col = torch.full((50, 1), max)

        left = temp[:, :curr_col+1]
        right = temp[:, curr_col+1:]
        temp = torch.cat((left, mean_col, min_col, max_col, right), dim=1)
    return temp

In [53]:
def forward_primitive(data_tensor, label, means, std_devs):
    params_dict = dict(net.named_parameters())
    
    print("Label: " + label)
    test_input = data_tensor
    inputs = test_input
    time1 = time.time()
    seq_len = inputs.size()[0] 
    
    zeroes = []
    zeroes.append([0.0 for j in range(100)]) # needs to match hidden_size of net
    h_t_minus_1 = copy.deepcopy(zeroes)
    h_t = copy.deepcopy(zeroes)

    inputs = inputs.tolist()
    for i in range(6):
        curr_col = i * 4
        col_values = []
        for row in range(50):
            col_values.append(inputs[row][curr_col])
        mean = sum(col_values) / 50
        minimum = min(col_values)
        maximum = max(col_values)
        for row in range(50):
            inputs[row].insert(curr_col+1, mean)
            inputs[row].insert(curr_col+2, minimum)
            inputs[row].insert(curr_col+3, maximum)
    
    rnn_input_weights = params_dict["rnn.weight_ih_l0"].tolist()
    rnn_hidden_weights = params_dict["rnn.weight_hh_l0"].tolist()
    rnn_input_biases = params_dict["rnn.bias_ih_l0"].tolist()
    rnn_hidden_biases = params_dict["rnn.bias_hh_l0"].tolist()
    for t in range(seq_len):
        input_t = inputs[t]
        for i in range(24):
            input_t[i] = input_t[i] - means[i]
            input_t[i] = input_t[i] / std_devs[i]
            
        in_mult_weights = multiply(input_t, transpose_matrix(rnn_input_weights))
        h_t_mult_weights = multiply(h_t_minus_1, transpose_matrix(rnn_hidden_weights))
        temp1 = add_vectors(in_mult_weights, h_t_mult_weights)
        temp2 = add_vectors(rnn_input_biases, rnn_hidden_biases)
        h_t = add_vectors(temp1, temp2)
        h_t = elementwise_tanh(h_t)
        h_t_minus_1 = h_t
        if t == 0:
            print("Sim debug buffer (partial):" + str(in_mult_weights[:10]))
        # below are also part of the debug buffer, excluded here for visibility
        #     print(h_t_mult_weights)
        #     print(temp1)
        #     print(rnn_input_biases)
        #     print(rnn_hidden_biases)
        #     print(temp2)
        #     print(h_t)
    
    rnn_output = h_t
    
    rnn_classi_weights = params_dict["classifier.weight"].tolist()
    rnn_classi_bias = params_dict["classifier.bias"].tolist()
    rnn_out_mult_weights = multiply(rnn_output, transpose_matrix(rnn_classi_weights))
    output = add_vectors(rnn_out_mult_weights, rnn_classi_bias)
    print("Sim output: " + str(output))
    
    max_conf = -9999
    index = 0
    max_conf_index = 0
    for conf in output:
        if (conf > max_conf):
            max_conf_index = index
            max_conf = conf
        index += 1
    prediction = max_conf_index
    print("Sim prediction: " + str(prediction))
    print("Sim prediction runtime: " + str(time.time() - time1))
    
    with torch.no_grad():
        time1 = time.time()
        expanded_input = add_sum_features(test_input)
        for row in expanded_input:
            for i in range(24):
                curr_data = row[i]
                curr_data = torch.sub(curr_data, means[i])
                curr_data = torch.div(curr_data, std_devs[i])
                row[i] = curr_data
        actual_output = net(expanded_input)
        _, predicted = torch.max(actual_output, 1)
        
        print("Actual output: " + str(actual_output))
        print("Actual prediction: " + str(predicted))
        print("Actual prediction runtime: " + str(time.time() - time1) + "\n")

means = [0.46261587738990784, 0.46261587738990784, -0.6956228613853455, 1.6631245613098145, -0.1328771561384201, -0.1328771412372589, -1.018193006515503, 0.631430447101593, -0.09139232337474823, -0.09139233082532883, -0.9628987908363342, 0.866577684879303, -6.184083461761475, -6.184083461761475, -192.46563720703125, 171.81434631347656, -1.4080398082733154, -1.408039927482605, -212.31764221191406, 193.5176239013672, 0.7322556972503662, 0.732255756855011, -164.01646423339844, 193.4847412109375]
std_devs = [1.0614013671875, 0.40623387694358826, 1.3376641273498535, 1.221408724784851, 0.8451341986656189, 0.5858472585678101, 1.2368916273117065, 0.8280805945396423, 1.1143925189971924, 0.874140202999115, 1.5301002264022827, 0.8200597763061523, 151.6263427734375, 25.81968116760254, 190.47702026367188, 165.2723846435547, 243.46754455566406, 29.674396514892578, 277.20880126953125, 262.9638366699219, 159.34500122070312, 30.97650909423828, 148.56300354003906, 194.8113555908203]
gestures = ["stationary", "random", "wave", "punch", "swipe", "wipe", "shake", "hammer"]
for gesture in gestures:
    test_filename = "test_" + gesture + "_input.txt"
        
    with open(test_filename, "r") as f:
        content = f.read()
        content_list = content.split("\n")
        shape = content_list[0]
        label = content_list[1]
        data = content_list[2]
        data = data[1:-1]
        data_array = np.empty((50, 7))
        index = 0
        for row_str in data.split('], ['):
            row_str = row_str.strip('[] ')
            row = [float(x) for x in row_str.split(', ')]
            data_array[index] =  np.array(row)
            index += 1
        data_tensor = torch.from_numpy(data_array)
        data_tensor = data_tensor.to(torch.float32)
        forward_primitive(data_tensor, label, means, std_devs)

Label: 0
Sim debug buffer (partial):[0.414622678611713, -0.06081006772415281, 0.4517822289211986, -0.058576991824177804, -0.2657550068098279, -0.025421263872602115, 0.39877024146280876, -0.010068004592120858, -0.4531850696439678, 0.2748962308075608]
Sim output: [6.345718582265222, 3.1781436974756616, -1.187690121588394, -0.8102586179991862, -2.756175875238431, -1.413985281636051, -2.945513135454442, -1.5735499657458567]
Sim prediction: 0
Sim prediction runtime: 0.25113773345947266
Actual output: tensor([[ 6.3457,  3.1781, -1.1877, -0.8103, -2.7562, -1.4140, -2.9455, -1.5735]])
Actual prediction: tensor([0])
Actual prediction runtime: 0.3084144592285156

Label: 1
Sim debug buffer (partial):[0.7090371814183664, -0.1332246480140507, 0.18923158418250563, 0.04594706438163598, -0.34590956248973825, 0.010461106241405074, 0.5117613852418765, 0.1936295102619912, -0.5396507231340169, 0.4252104852782542]
Sim output: [5.583307187678921, 3.7533906945798603, -2.9733742071443325, -1.5730900478081964,

In [63]:
# MIT License
#
# Copyright (c) 2018 Stefano Nardo https://gist.github.com/stefanonardo
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [65]:
def add_sum_features(input):
    temp = input
    for i in range(6):
        curr_col = i * 4
        mean = torch.mean(temp[:, curr_col]).item()
        min = torch.min(temp[:, curr_col]).item()
        max = torch.max(temp[:, curr_col]).item()
        mean_col = torch.full((50, 1), mean)
        min_col = torch.full((50, 1), min)
        max_col = torch.full((50, 1), max)

        left = temp[:, :curr_col+1]
        right = temp[:, curr_col+1:]
        temp = torch.cat((left, mean_col, min_col, max_col, right), dim=1)
    return temp

In [69]:
def train(config, dataset):
    es = EarlyStopping(patience=10)
    net = Net(config["hidden_size"])

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=config["lr"], weight_decay=0.01)

    trainloader, validloader = create_dataloaders(dataset, config["batch_size"])

    can_print = True
    for epoch in range(config["max_num_epochs"]):
        val_loss = 0.0
        total = 0
        correct = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = torch.squeeze(inputs)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            
            if (len(outputs.size()) > 2): # squeeze if batch has more than 1 datapoint
                outputs = torch.squeeze(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            for data in validloader:
                inputs, labels = data
                inputs = torch.squeeze(inputs)
                outputs = net(inputs)
                if (len(outputs.size()) > 2):
                    outputs = torch.squeeze(outputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            path = os.path.join(temp_checkpoint_dir, "checkpoint.pt")
            torch.save((net.state_dict()), path)
            checkpoint = tune.Checkpoint.from_directory(temp_checkpoint_dir)
            
            tune.report(
                {"loss": val_loss / len(validloader), "accuracy": correct / total}, checkpoint = checkpoint
            )

        if es.step(val_loss):
            break

In [71]:
def main(num_samples=160, max_num_epochs=100):
    config = {
        #"batch_size": tune.choice([2, 4, 8, 16, 32, 64]),
        "batch_size": 32,
        #"hidden_size": tune.choice([20 * i for i in range(1, 11)]),
        "hidden_size": 100,
        #"lr": tune.choice([0.1, 0.01, 0.001, 0.0001]),
        "lr": 0.001,
        "max_num_epochs": max_num_epochs
    }
    
    scheduler = ASHAScheduler(
        time_attr="training_iteration",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

    dataset = CustomDataset("labels.csv", "train_data")
    
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train, dataset=dataset),
            resources={"cpu": 1}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            #scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    results = tuner.fit()
    
    best_result = results.get_best_result()

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    return best_result, dataset.means, dataset.std_devs

In [73]:
def test_model(net, log_file, means, std_devs):    
    gestures = ["stationary", "random", "wave", "punch", "swipe", "wipe", "shake", "hammer"]
    for gesture in gestures:
        datafile = gesture + "_labels.csv"
        dataset = CustomDataset(datafile, "test_data", means, std_devs)
        testloader = DataLoader(dataset, batch_size=5, shuffle=True)
        
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                inputs = torch.squeeze(inputs)
                outputs = net(inputs)
                if (len(outputs.size()) > 2): # squeeze if batch has more than 1 datapoint
                    outputs = torch.squeeze(outputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        log = f'Accuracy of the network on {gesture}: {100 * correct // total}%\n'
        with open(log_file, "a") as f:
            f.write(log)

os.chdir("C:\\Users\\tjjtj\\CG4002\\glove_data")
log_file_basename = "log"
extension = ".txt"
counter = 1
while True:
    log_file = f"{log_file_basename}_{counter}{extension}"
    if not os.path.exists(log_file):
        break
    counter += 1

for i in range(20):
    best_result, means, std_devs = main(num_samples=20, max_num_epochs=100)    
    best_trained_model = Net(best_result.config["hidden_size"])
    
    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")
    model_state = torch.load(checkpoint_path, weights_only=True)
    best_trained_model.load_state_dict(model_state)
    save_basename = "saved_model"
    save_ext = ".pt"
    counter = 1
    while True:
        save_file = f"{save_basename}_{counter}{save_ext}"
        if not os.path.exists(save_file):
            break
        counter += 1
    torch.save(best_trained_model.state_dict(), save_file)

    with open(log_file, "a") as f:
        f.write(f"Model {i}\n")
        f.write(f"Means {means}\n")
        f.write(f"Std devs {std_devs}\n")
    test_model(best_trained_model, log_file, means, std_devs)
    with open(log_file, "a") as f:
        f.write("\n")

(train pid=524) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/tjjtj/ray_results/train_2025-11-15_16-23-57/train_8d90d_00001_1_2025-11-15_16-24-50/checkpoint_000000)
(train pid=35312) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/tjjtj/ray_results/train_2025-11-15_16-23-57/train_8d90d_00018_18_2025-11-15_16-24-50/checkpoint_000000) [repeated 15x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
2025-11-15 16:26:26,118	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(train pid=35132) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/tjjtj/ray_results/train_20

Best trial config: {'batch_size': 32, 'hidden_size': 100, 'lr': 0.001, 'max_num_epochs': 100}
Best trial final validation loss: 0.22854599748787127
Best trial final validation accuracy: 0.9111111111111111
